<a href="https://colab.research.google.com/github/AI4Finance-Foundation/FinRL/blob/master/FinRL_Raytune_for_Hyperparameter_Optimization_RLlib%20Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#Installing FinRL
%%capture
#!pip install git+https://github.com/AI4Finance-LLC/FinRL-Library.git

UsageError: Line magic function `%%capture` not found.


In [6]:
%%capture
#!pip install "ray[tune]" optuna

In [7]:
%%capture
#!pip install int_date==0.1.8

In [1]:
%reload_ext autoreload
%autoreload 2

#Importing libraries

In [2]:
#Importing the libraries
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime
import optuna
%matplotlib inline
from finrl import config
from finrl.finrl_meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.finrl_meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.finrl_meta.env_stock_trading.env_stocktrading_np import StockTradingEnv as StockTradingEnv_numpy 
from finrl.agents.rllib.models import DRLAgent as DRLAgent_rllib
from stable_baselines3.common.vec_env import DummyVecEnv
from finrl.finrl_meta.data_processor import DataProcessor
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
import ray
from pprint import pprint
from ray.rllib.agents.ppo import PPOTrainer
from ray.rllib.agents.ddpg import DDPGTrainer
from ray.rllib.agents.a3c import A2CTrainer
from ray.rllib.agents.a3c import a2c
from ray.rllib.agents.ddpg import ddpg, td3
from ray.rllib.agents.ppo import ppo
from ray.rllib.agents.sac import sac
import sys
sys.path.append("../FinRL-Library")
import os
import itertools
from ray import tune
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.optuna import OptunaSearch

from ray.tune.registry import register_env

import time
import psutil
psutil_memory_in_bytes = psutil.virtual_memory().total
ray._private.utils.get_system_memory = lambda: psutil_memory_in_bytes
from typing import Dict, Optional, Any

/home/mohammad/anaconda3/envs/FinRl/lib/python3.8/site-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  warnings.warn(


In [3]:
import os
root_path = 'MARKETS'
from finrl import config
from finrl import config_tickers
import os
from finrl.main import check_and_make_directories

CHOOSEN_MARKET = 'Crypto_market' 

DATA_SAVE_DIR = os.path.join(root_path, CHOOSEN_MARKET, 'DATASET')
TRAINED_MODEL_DIR = os.path.join(root_path, CHOOSEN_MARKET, 'TRAINED_MODEL_DIR')
TENSORBOARD_LOG_DIR = os.path.join(root_path, CHOOSEN_MARKET, 'TENSORBOARD_LOG_DIR')
RESULTS_DIR = os.path.join(root_path, CHOOSEN_MARKET, 'RESULTS_DIR')
TUNED_MODELS_DIR = os.path.join(root_path, CHOOSEN_MARKET, 'TUNED_MODELS')

check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR, TUNED_MODELS_DIR])

#if not os.path.exists("./" + config.DATA_SAVE_DIR):
#    os.makedirs("./" + config.DATA_SAVE_DIR)
#if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
#    os.makedirs("./" + config.TRAINED_MODEL_DIR)
#if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
#    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
#if not os.path.exists("./" + config.RESULTS_DIR):
#    os.makedirs("./" + config.RESULTS_DIR)
# if not os.path.exists("./" + "tuned_models"):
#     os.makedirs("./" + "tuned_models")

##Defining the hyperparameter search space

1. You can look up [here](https://docs.ray.io/en/latest/tune/key-concepts.html#search-spaces) to learn how to define hyperparameter search space
2. Jump over to this [link](https://github.com/DLR-RM/rl-baselines3-zoo/blob/master/utils/hyperparams_opt.py) to find the range of different hyperparameter
3. To learn about different hyperparameters for different algorithms for RLlib models, jump over to this [link](https://docs.ray.io/en/latest/rllib-algorithms.html)

In [4]:
def sample_ddpg_params():
  
  return {
  "buffer_size": tune.choice([int(1e4), int(1e5), int(1e6)]),
  "lr": tune.loguniform(1e-5, 1),
  "train_batch_size": tune.choice([32, 64, 128, 256, 512])
  }
def sample_a2c_params():
  
  return{
       "lambda": tune.choice([0.1,0.3,0.5,0.7,0.9,1.0]),
      "entropy_coeff": tune.loguniform(0.00000001, 0.1),
      "lr": tune.loguniform(1e-5, 1) 
      
  }

def sample_ppo_params():
  return {
      "entropy_coeff": tune.loguniform(0.00000001, 0.1),
      #"entropy_coeff": tune.loguniform(0.0001, 0.1),
      "lr": tune.loguniform(5e-5, 1),
      #"lr": tune.loguniform(5e-4, 1),
      "sgd_minibatch_size": tune.choice([ 32, 64, 128, 256, 512]),
      "lambda": tune.choice([0.1,0.3,0.5,0.7,0.9,1.0])
  }
  

In [5]:
MODELS = {"a2c": a2c, "ddpg": ddpg, "td3": td3, "sac": sac, "ppo": ppo}

## Getting the training and testing environment

In [17]:
from finrl.finrl_meta.preprocessor.CryptoDataReader import CryptoDataLoader
import datetime
from finrl.finrl_meta.env_stock_trading.env_stocktrading import StockTradingEnv


In [7]:
api_key='cbj22uqad3i2thcmtg80'
choosen_symbols = ['BTC', 'ETH', 'USDT', 'USDC', 'BNB', 'XRP', 'ADA', 'BUSD', 'SOL', 'DOT']

In [8]:
def fix_date(crypto_df):
    dates = []
    for unixdate in crypto_df['date']:
        utcdate = datetime.datetime.fromtimestamp(unixdate).strftime('%Y-%m-%d')
        dates.append(utcdate)
    crypto_df['date'] = dates
    return crypto_df

def fetch_data(API_KEY, choosen_symbols, start, end):
    data_loader = CryptoDataLoader(choosen_symbols, API_KEY)
    crypto_df = data_loader.load_crypto_candles(start= start, end= end)
    crypto_df = data_loader.cleansing_holcv_dataframes(crypto_df)
    crypto_df = fix_date(crypto_df)
    data = crypto_df.copy()
    data  = data_loader.crypto_clean_data(data)
    
    return data


In [18]:
def config_env_kwargs(train):
    stock_dimension = len(train.tic.unique())
    state_space = 1 + len(config.INDICATORS)*stock_dimension + 2*stock_dimension
    print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")
    
    buy_cost_list = sell_cost_list = [0.001] * stock_dimension
    num_stock_shares = [0] * stock_dimension

    env_kwargs = {
        "hmax": 100,
        "initial_amount": 1000000,
        "num_stock_shares": num_stock_shares,
        "buy_cost_pct": buy_cost_list,
        "sell_cost_pct": sell_cost_list,
        "state_space": state_space,
        "stock_dim": stock_dimension,
        "tech_indicator_list": config.INDICATORS,
        "action_space": stock_dimension,
        "reward_scaling": 1e-4
    }
    return env_kwargs

In [19]:
def fetch_preprocess_data(API_KEY,start_date, end_date, choosen_symbols, 
          technical_indicator_list):
    
    #fetch data
    #DP = DataProcessor(data_source, **kwargs)
    #data = DP.download_data(ticker_list, start_date, end_date, time_interval)
    #data = DP.clean_data(data)
    #data = DP.add_technical_indicator(data, technical_indicator_list)
    #if if_vix:
    #    data = DP.add_vix(data)
    #price_array, tech_array, turbulence_array = DP.df_to_array(data, if_vix)

    data = fetch_data(API_KEY, choosen_symbols, start_date, end_date)

    
    fe = FeatureEngineer(
                    use_technical_indicator = True,
                    tech_indicator_list =  config.INDICATORS,
                    use_vix= True ,
                    use_turbulence=True,
                    user_defined_feature = False)

    processed = fe.preprocess_data(data)

    list_ticker = processed["tic"].unique().tolist()
    list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
    combination = list(itertools.product(list_date,list_ticker))
    
    processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
    processed_full = processed_full[processed_full['date'].isin(processed['date'])]
    processed_full = processed_full.sort_values(['date','tic'])

    processed_full = processed_full.fillna(0)

    train_start_date = datetime.datetime(2019,10,21).strftime('%Y-%m-%d')
# from config.py TRAIN_END_DATE is a string
    train_end_date = datetime.datetime(2021,4,24).strftime('%Y-%m-%d')
    trade_end_date = datetime.datetime(2021,9,24).strftime('%Y-%m-%d')
    train = data_split(processed_full, train_start_date ,train_end_date)
    trade = data_split(processed_full, train_end_date ,trade_end_date)

    return train, trade

def get_train_env(train, env_kwargs):
    e_train_gym = StockTradingEnv(df = train, **env_kwargs)
    return e_train_gym

In [20]:
#Function to calculate the sharpe ratio from the list of total_episode_reward
def calculate_sharpe(episode_reward:list):
  perf_data = pd.DataFrame(data=episode_reward,columns=['reward'])
  perf_data['daily_return'] = perf_data['reward'].pct_change(1)
  if perf_data['daily_return'].std() !=0:
    sharpe = (252**0.5)*perf_data['daily_return'].mean()/ \
          perf_data['daily_return'].std()
    return sharpe
  else:
    return 0

def get_test_config(trade, env_kwargs):
  
  ##DP = DataProcessor(data_source, **kwargs)
  ##data = DP.download_data(ticker_list, start_date, end_date, time_interval)
  ##data = DP.clean_data(data)
  ##data = DP.add_technical_indicator(data, technical_indicator_list)
  
  ##if if_vix:
  ##    data = DP.add_vix(data)
  
  ##price_array, tech_array, turbulence_array = DP.df_to_array(data, if_vix)
  ##test_env_config = {'price_array':price_array,
  ##          'tech_array':tech_array,
  ##          'turbulence_array':turbulence_array,'if_train':False}
  ##return test_env_config
  
  e_trade_gym = StockTradingEnv(df = trade,  turbulence_threshold = 70,risk_indicator_col='vix', **env_kwargs)


def val_or_test(test_env_config,agent_path,model_name,env):
  episode_total_reward = DRL_prediction(model_name,test_env_config,
                                env = env,
                                agent_path=agent_path)


  return calculate_sharpe(episode_total_reward),episode_total_reward

In [21]:
TRAIN_START_DATE = datetime.date(2019,10,21).timetuple()
TRAIN_END_DATE = datetime.date(2020,10,21).timetuple()

VAL_START_DATE = datetime.date(2020,10,22).timetuple()
VAL_END_DATE = datetime.date(2021,4,24).timetuple()

TEST_START_DATE = datetime.date(2021,4,25).timetuple()
TEST_END_DATE = datetime.date(2021,9,24).timetuple()

In [22]:
technical_indicator_list =config.INDICATORS
model_name = 'ppo'


In [23]:
technical_indicator_list

['macd',
 'boll_ub',
 'boll_lb',
 'rsi_30',
 'cci_30',
 'dx_30',
 'close_30_sma',
 'close_60_sma']

In [24]:
api_key='cbj22uqad3i2thcmtg80'
choosen_symbols = ['BTC', 'ETH', 'USDT', 'USDC', 'BNB', 'XRP', 'ADA', 'BUSD', 'SOL', 'DOT']

In [25]:
train,trade = fetch_preprocess_data(api_key,TRAIN_START_DATE,TEST_END_DATE,choosen_symbols, technical_indicator_list)


704


/mnt/f/financial_projects/Deep Reinforcement Learning Approaches on Stock Prediction/FinRL/finrl/finrl_meta/preprocessor/preprocessors.py:151: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indicator_df.append(
/mnt/f/financial_projects/Deep Reinforcement Learning Approaches on Stock Prediction/FinRL/finrl/finrl_meta/preprocessor/preprocessors.py:151: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indicator_df.append(
/mnt/f/financial_projects/Deep Reinforcement Learning Approaches on Stock Prediction/FinRL/finrl/finrl_meta/preprocessor/preprocessors.py:151: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indicator_df.append(
/mnt/f/financial_projects/Deep Reinforcement Learning Appr

Successfully added technical indicators
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (279, 8)
Successfully added vix


/mnt/f/financial_projects/Deep Reinforcement Learning Approaches on Stock Prediction/FinRL/finrl/finrl_meta/preprocessor/yahoodownloader.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_df = data_df.append(temp_df)


Successfully added turbulence index


## Registering the environment

In [26]:
from ray.tune.registry import register_env

env_kwargs = config_env_kwargs(train)
#get_train_env(train, env_kwargs)
env_name = 'StockTrading_train_env'
register_env(env_name, lambda config: get_train_env(train, env_kwargs))

Stock Dimension: 34, State Space: 341


In [27]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [28]:
torch.cuda.is_available()

True

## Running tune 

In [30]:
MODEL_TRAINER = {'a2c':A2CTrainer,'ppo':PPOTrainer,'ddpg':DDPGTrainer}
if model_name == "ddpg":
    sample_hyperparameters = sample_ddpg_params()
elif model_name == "ppo":
  sample_hyperparameters = sample_ppo_params()
elif model_name == "a2c":
  sample_hyperparameters = sample_a2c_params()
  
def run_optuna_tune():

  algo = OptunaSearch()
  algo = ConcurrencyLimiter(algo,max_concurrent=4)
  scheduler = AsyncHyperBandScheduler()
  num_samples = 10
  training_iterations = 100

  analysis = tune.run(
      MODEL_TRAINER[model_name],
      metric="episode_reward_mean", #The metric to optimize for tuning
      mode="max", #Maximize the metric
      search_alg = algo,#OptunaSearch method which uses Tree Parzen estimator to sample hyperparameters
      scheduler=scheduler, #To prune bad trials
      config = {**sample_hyperparameters,
                'env':'StockTrading_train_env','num_workers':1,
                'num_gpus':1,'framework':'torch'},
      num_samples = num_samples, #Number of hyperparameters to test out
      stop = {'training_iteration':training_iterations},#Time attribute to validate the results
      verbose=1,local_dir=TUNED_MODELS_DIR,#Saving tensorboard plots
      # resources_per_trial={'gpu':1,'cpu':1},
      max_failures = 1,#Extra Trying for the failed trials
      raise_on_failed_trial=False,#Don't return error even if you have errored trials
      keep_checkpoints_num = num_samples-5, 
      checkpoint_score_attr ='episode_reward_mean',#Only store keep_checkpoints_num trials based on this score
      checkpoint_freq=training_iterations#Checpointing all the trials
  )
  print("Best hyperparameter: ", analysis.best_config)
  return analysis

In [31]:
analysis = run_optuna_tune()

Trial name,# failures,error file
PPOTrainer_StockTrading_train_env_60e92f84,2,"/mnt/f/financial_projects/Deep Reinforcement Learning Approaches on Stock Prediction/FinRL/MARKETS/Crypto_market/TUNED_MODELS/PPOTrainer_2022-10-08_19-59-27/PPOTrainer_StockTrading_train_env_60e92f84_1_entropy_coeff=0.0035,env=StockTrading_train_env,framework=torch,lambda=0.7000,lr=0.02_2022-10-08_19-59-28/error.txt"
PPOTrainer_StockTrading_train_env_852ee1f4,2,"/mnt/f/financial_projects/Deep Reinforcement Learning Approaches on Stock Prediction/FinRL/MARKETS/Crypto_market/TUNED_MODELS/PPOTrainer_2022-10-08_19-59-27/PPOTrainer_StockTrading_train_env_852ee1f4_2_entropy_coeff=0.0000,env=StockTrading_train_env,framework=torch,lambda=0.9000,lr=0.08_2022-10-08_20-00-54/error.txt"
PPOTrainer_StockTrading_train_env_e5b5ec2a,1,"/mnt/f/financial_projects/Deep Reinforcement Learning Approaches on Stock Prediction/FinRL/MARKETS/Crypto_market/TUNED_MODELS/PPOTrainer_2022-10-08_19-59-27/PPOTrainer_StockTrading_train_env_e5b5ec2a_3_entropy_coeff=0.0007,env=StockTrading_train_env,framework=torch,lambda=0.7000,lr=0.60_2022-10-08_20-03-57/error.txt"


## Best config, directory and checkpoint for hyperparameters



In [23]:
best_config = analysis.get_best_config(metric='episode_reward_mean',mode='max')
best_config

2022-10-08 16:14:07,948	WARNING experiment_analysis.py:591 -- Could not find best trial. Did you pass the correct `metric` parameter?


In [24]:
best_logdir = analysis.get_best_logdir(metric='episode_reward_mean',mode='max')
best_logdir

2022-10-08 16:14:21,860	WARNING experiment_analysis.py:591 -- Could not find best trial. Did you pass the correct `metric` parameter?


In [ ]:
best_checkpoint = analysis.best_checkpoint
best_checkpoint

'/content/tuned_models/A2C_2022-01-06_06-46-44/A2C_StockTrading_train_env_c4a4a4e2_7_entropy_coeff=1.9875e-06,framework=torch,lambda=0.5,lr=0.0034153,num_gpus=1,num_workers=1_2022-01-06_07-25-04/checkpoint_000100/checkpoint-100'

In [ ]:
# sharpe,df_account_test,df_action_test = val_or_test(TEST_START_DATE, TEST_END_DATE, ticker_list, data_source, time_interval, 
#          technical_indicator_list, env, model_name,best_checkpoint, if_vix = True)

In [ ]:
test_env_config = get_test_config(TEST_START_DATE, TEST_END_DATE, ticker_list, data_source, time_interval, 
                        technical_indicator_list, env, model_name)

[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (294, 9)
Clean data for TSLA
Data clean for TSLA is finished.
Data clean all finished!
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (294, 9)
Clean data for ^VIX
Data clean for ^VIX is finished.
Data clean all finished!
['TSLA']
Successfully transformed into array


In [ ]:
sharpe,account,actions = val_or_test(test_env_config,agent_path,model_name,env)

In [ ]:
def DRL_prediction(
        model_name,
        test_env_config,
        env,
        model_config,
        agent_path,
        env_name_test='StockTrading_test_env'
    ):

        env_instance = env(test_env_config)
        
        register_env(env_name_test, lambda config: env(test_env_config))
        model_config['env'] = env_name_test
        # ray.init() # Other Ray APIs will not work until `ray.init()` is called.
        if model_name == "ppo":
            trainer = MODELS[model_name].PPOTrainer(config=model_config)
        elif model_name == "a2c":
            trainer = MODELS[model_name].A2CTrainer(config=model_config)
        elif model_name == "ddpg":
            trainer = MODELS[model_name].DDPGTrainer(config=model_config)
        elif model_name == "td3":
            trainer = MODELS[model_name].TD3Trainer(config=model_config)
        elif model_name == "sac":
            trainer = MODELS[model_name].SACTrainer(config=model_config)

        try:
            trainer.restore(agent_path)
            print("Restoring from checkpoint path", agent_path)
        except BaseException:
            raise ValueError("Fail to load agent!")

        # test on the testing env
        state = env_instance.reset()
        episode_returns = list()  # the cumulative_return / initial_account
        episode_total_assets = list()
        episode_total_assets.append(env_instance.initial_total_asset)
        done = False
        while not done:
            action = trainer.compute_single_action(state)
            state, reward, done, _ = env_instance.step(action)

            total_asset = (
                env_instance.amount
                + (env_instance.price_ary[env_instance.day] * env_instance.stocks).sum()
            )
            episode_total_assets.append(total_asset)
            episode_return = total_asset / env_instance.initial_total_asset
            episode_returns.append(episode_return)
        ray.shutdown()
        print("episode return: " + str(episode_return))
        print("Test Finished!")
        return episode_total_assets

In [ ]:
episode_total_assets = DRL_prediction(
        model_name,
        test_env_config,
        env,
        best_config,
        best_checkpoint,
        env_name_test='StockTrading_test_env')

(RolloutWorker pid=54558) 2022-01-06 08:15:05,638	WARNING deprecation.py:46 -- DeprecationWarning: `convert_to_non_torch_type` has been deprecated. Use `ray/rllib/utils/numpy.py::convert_to_numpy` instead. This will raise an error in the future!
2022-01-06 08:15:05,784	INFO trainable.py:468 -- Restored on 172.28.0.2 from checkpoint: /content/tuned_models/A2C_2022-01-06_06-46-44/A2C_StockTrading_train_env_c4a4a4e2_7_entropy_coeff=1.9875e-06,framework=torch,lambda=0.5,lr=0.0034153,num_gpus=1,num_workers=1_2022-01-06_07-25-04/checkpoint_000100/checkpoint-100
2022-01-06 08:15:05,857	INFO trainable.py:475 -- Current state after restoring: {'_iteration': 100, '_timesteps_total': 0, '_time_total': 1010.0849390029907, '_episodes_total': 307}


Restoring from checkpoint path /content/tuned_models/A2C_2022-01-06_06-46-44/A2C_StockTrading_train_env_c4a4a4e2_7_entropy_coeff=1.9875e-06,framework=torch,lambda=0.5,lr=0.0034153,num_gpus=1,num_workers=1_2022-01-06_07-25-04/checkpoint_000100/checkpoint-100
episode return: 1.0
Test Finished!


In [ ]:
print('The test sharpe ratio is: ',calculate_sharpe(episode_total_assets))
df_account_test = pd.DataFrame(data=episode_total_assets,columns=['account_value'])